#### ------------------------------------------------------------ WORK IN PROGRESS ------------------------------------------------------------------------------------------
# Credits
Some space credits to [sources](https://stackexchange.com/). 

### Authors: Fernando Lasso
### Project: Seminar Case Studies ESE Bachelor Econometrics
### Project members: TBA

# Abstract

This code has been written for a case study exploring the research question "Do volatility modelled portfolios ourperform the naive strategy".

# Objective:

A notebook providing the code used to calculate the returns of a monthly rebalanced portfolio.

**Table of content:**

1. [Libraries and data](#1-bullet)
2. [Data enrichment](#2-bullet)
3. [Portfolio creation and evaluation](#3-bullet)
4. [Benchmark strategies](#4-bullet)
5. [GARCH Theory](#5-bullet)
6. [GARCH Performance](#6-bullet)
7. [Strategies over time](#7-bullet)


 # 1. Library and data:<a class="anchor" id="1-bullet"></a>
 
 Below, a list of libraries necessary for the calculations is loaded.

In [ ]:
# Read all libraries
packages = c(
             "PerformanceAnalytics",
             #"e1071" ,
             #"nlme",
             #"SpatioTemporal",
             "readxl",
             #"timeSeries",
             #"fGarch",
             "dplyr",
             #"zoo",
             #"assert_that",
             "ggplot2",
             "reshape2",
             #"quantmod",
             "quadprog",
             "lubridate",
             "foreach",
             #"caret",
             #"car",
             "Matrix",
             "logOfGamma",
             "tseries") 


for (m in 1:length(packages)){
  cat("Loading package: ",packages[m],"\n")
  suppressMessages(library(packages[m],character.only=TRUE))
}

cat("... finished loading packages \n")

The data consists of daily simple returns of five factors over 657 months, from July 1963 until March 2018.
These are based on the [Fama and French](https://www.jstor.org/stable/2329302) Three Factor Model and the more recently their [Five Factor Model](https://www.sciencedirect.com/science/article/pii/S0304405X15000616). The factors used in the Three Factor Model are: the market factor in excess over the risk-free return ($\text{MKT} - r_f$), a size factor SMB (small minus big) and a value factor HML (high minus low). The size factor is constructed by taking a long position in small stocks and short position in big stocks, while the value factor HML encompasses the difference between returns of high and low book-to-market ratio firms.

Criticism on exclusion of profitability and investment based portfolios led to the construction of the Five Factor Model, which extends the Three factor model by including a momentum profitability factor RMW (robust minus weak) and investment factor CMA (conservative minus aggressive). Except for the market factor, all factors are zero-investment portfolios.

In [ ]:
# Read excel
data <- read_excel("../input/famadata/F-F_Research_Data_5_Factors_2x3_daily.xlsx",skip = 2)
recessions <- read_excel("../input/famadata/Recessions.xlsx",skip = 10)
print(summary(data))

As you can see, the information on returns runs from half 1963 till the beginning of 2018. The returns are expressed as daily percentages and corrected for the risk free rate. Note that a mean return of 0.025 means that on average, 0.025 percent return is expected per day when investing in this portfolio. The cumulative return for one portfolio over all of the 54 years would thus be the product of its rows. For the market porfolio, this means that if you would have invested one dollar in 1963, it would now be worth:

In [ ]:
prod(1+ data$`Mkt-RF`/100 )

 # 2. Data Enrichment:<a class="anchor" id="2-bullet"></a>
 
 Next, some feature engineering from the available information. Also, only a subset from the data will be taken to speed up the run time.

In [ ]:
x <- 0.3

In [ ]:
#Take last x percent of data
data <- tail(data, nrow(data)*x)

# Correct the date format
data$Date <-  as.Date(as.character(data$X__1), format="%Y%m%d")

# Get year 
data$year <- year(data$Date)

# Get month
data$month <- month(data$Date)

From the Recessions data, we will add a column to our dataframe containing a dummy for recessions

In [ ]:
# Get Recession dummy
recessions$year <- substr(recessions[[1]],1,4)
recessions$month <- substr(recessions[[1]],6,7)
recessions$month[substr(recessions$month,1,1)==0] <- substr(recessions$month[substr(recessions$month,1,1)==0] ,2,2)

data <- merge(data,recessions, by = c("year","month"),all.x=T)

Since our portfolio strategies will choose from these factors, we will create a new dataframe to pass as an input to our functions.

In [ ]:
inputData <- data[,c("Mkt-RF","SMB","HML","RMW","CMA")]

# 3. Portfolio creation and evaluation: <a class="anchor" id="3-bullet"></a>

In order to re-balance the portfolio at the end of every month, it is necessary to evaluate the performance of the individual factors throughout the month. At the start of each month, wealth is either allocated equally $1/N$ across the factors, inversely proportional to the previous month’s volatility (managed volatility) or inversely proportional to the forecasted volatility (modelled volatility). After each subsequent day, the current weight of each factor $i$ in the portfolio is re-calculated based on the returns of the previous day:

![ImgurF1](https://i.imgur.com/gz6VPaV.png)

where $m$ denotes the number of days in the current month. In **(1)**, dividing by the term  $\sum_{j=1}^5 w_{t+d-1}^{(j)} (1 + r_{t+d-1}^{(j)})$ ensures that the weights assigned to each factor sum to one at all times. Then, the daily return of the entire portfolio is found by taking a weighted average of the individual factor returns, where weights are computed as in **(1)**. Finally, the portfolio is re-balanced at the end of the month based on the end-of-month wealth and choice of weighting rule used. The end-of-month wealth $W_{t+m}^{(P)}$ is determined by multiplying the previous month’s wealth $W_t^{(P)}$ by the cumulative portfolio return over the entire month: 

Click on the *code* button below to see the function that will loop through our portfolio returns and rebalance the chosen weights. 


In [ ]:
# Evaluate performance of a strategy function on a portfolio given the rebalancing frequency
portfolio_returns <- function(portfolio, order.by, strategy, window = NULL, target = NULL){
    # Get number of rows, set starting weight and returns
    n <- nrow(portfolio)
    weights <- rep(1,m)
    weights <- weights/sum(weights)
    returns <- rep(0,n)


    # Progress bar because why not
    pb <- txtProgressBar(title = "progress bar", min = 0,
                       max = n, width = 100)

    # Save first of period
    first_of_period <- 1

    # Count days in month
    period <- unlist(data %>% group_by(year,month) %>% mutate(period_n = n()) %>% ungroup() %>% dplyr::select(period_n))
    
    # Loop over rows
    for(t in 1:n){
        if(t!=1){
            # Every time a new period begins, rebalance the portfolio weights
            if(order.by[t]!=order.by[t-1]){
                # Strategy used, number of assets, give all past returns
                weights <- getweights(strategy = strategy, returns = head(portfolio,(t-1)), 
                                      last_period_n = (t-first_of_period), next_period_n = period[t],
                                      window = window, target = target) 
                # Update first of period
                first_of_period <- 1
                # Sanity check to make sure weights sum up to one (approximately, allowing for rounding error)
                if(abs(sum(weights)-1)>1e-05){ print(weights); abortmission}
            }
    }
    
    # Get todays returns
    day_returns <-portfolio[t,]
    
    # Calculate return from this day from the current weights. Scale weights to 1 and then multiply with percentages.
    returns[t] <- sum((weights/abs(sum(weights))) * ((day_returns/100)))

    # Recalculate weights
    weights <-  weights * (1+(day_returns/100))
    
    setTxtProgressBar(pb, t, label=paste( round(i/n*100, 0),"% done"))
  }
  close(pb)
  return((returns)*100)
}

## Performance measures

Need some metrics to evaluate performance of portfolio. After calculating factor weights and the corresponding returns, performance measures are used to make comparisons between models, both in the full sample and in various sub-samples. Five performance measures are computed: the Sharpe and Sortino ratios,  value-at-risk (VaR), skewness and Jensen's alpha. The annualized Sharpe and Sortino ratios are given by the following formulas: 

![F2](https://i.imgur.com/9Hq5IVs.png)

where $\overline{r}=\sum_{t=1}^{T}r_t$ denotes the average daily return of the portfolio. These measures calculate the risk-adjusted performance of a portfolio and therefore a higher score indicates better performance. The advantage of the Sortino ratio over the more common Sharpe ratio is that volatility associated with large positive returns is not penalized by the measure. VaR, on the other hand,  calculates the maximum loss over a defined period at a 95\% confidence level: see equation (\ref{eq:ValueatRisk}). The closer this value is to zero, the better the performance:

![F3](https://i.imgur.com/CZZ1exp.png)

Furthermore, the skewness of the return distribution is assessed. A positive skewness indicates a favourable investment, as gains occur more frequently than losses. It is calculated as the third moment of the empirical distribution. Finally, Jensen's alpha is also considered, which measures the return of a portfolio is excess of the theoretical expected return. 

When comparing models, one is said to outperform another if its performance measures, in particular the Sortino and Sharpe ratios, are more favorable. 

In [ ]:
getPerfomances <- function(columns){

    ###### PERFORMANCE MEASURES #######
    melt.data <- melt(data,id.vars = c("year","Mkt-RF"),measure.vars = columns)

    performances <- melt.data %>% group_by(variable) %>% na.omit() %>% dplyr::summarise(Mean = mean(value), 
                                                                                     STD = sd(value),
                                                                                     Sharpe = Mean/STD * sqrt(252),
                                                                                     Sortino = SortinoRatio(value)* sqrt(252),
                                                                                     VAR = VaR(value/100, method="modified"),
                                                                                     Skew = skewness(value),
                                                                                     Jensen = getJensen(value,`Mkt-RF`)
                                                                                     )    
    return(performances)
}

#Calculate Jensen of one column versus the market minus RF
getJensen <- function(asset,market){
    fit <- lm(asset ~ market)
    return(fit$coefficients[1])
}

# 4. Benchmark strategies: <a class="anchor" id="4-bullet"></a>

## Naive portfolio

Naive strategy simply rebalances the investment to be equally spread over N assets.. In other words, it weighs each asset equally 1 over N.

In [ ]:
# Rebalance weights based on underlying portfolio inputs and strategy
getweights <- function(strategy, returns, last_period_n, next_period_n, window=NULL, target=NULL){

    # Get number of assets in portfolio
    m <- ncol(returns)
    
    # Naive strategy gives equal weights to all assets
    if(strategy == "naive"){
        return(rep(1/m, m))
    }
}

Now, let's calculate the returns of the monthly rebalanced naive portfolio and see a quick summary.

In [ ]:
# Get returns as percentages
data$Naive <- portfolio_returns(inputData,data$month,"naive") 
summary(data$Naive)

## Volatility Managed portfolio portfolio

 Volatility managed strategy allocates weights inversely proportional to the historical variance of the single assets. The forecasted variance for day $t+d$ is calculated as the average volatility of the previous period.

In [ ]:
# Rebalance weights based on underlying portfolio inputs and strategy
getweights <- function(strategy, returns, last_period_n, next_period_n, window=NULL, target=NULL){

    # Get number of assets in portfolio
    m <- ncol(returns)
    
    # Volatility managed strategy gives weights inversely proportional to last month's volatility
    if(strategy == "vol"){
        lastPeriod <- tail(returns, last_period_n)
    
        #Initialize the sigma squared variable, denoting volatility
        sig2 <- rep(0,last_period_n)
        
        # Get last month's mean returns
        mean_ret <- colMeans(lastPeriod)
        
        # See formula
        sig2 <- colSums(sweep(lastPeriod,2,mean_ret) * sweep(lastPeriod,2,mean_ret)) / last_period_n
              
        # Set weights inversely proportional to variance
        w <- 1/(sig2)
        
        # Scale sum of weights to one
        return(w/abs(sum(w)))
    }
}

Let's observe this portfolio's performance.'

In [ ]:
data$VolMan <- portfolio_returns(inputData,data$month,"vol") 
summary(data$VolMan)

## Mean Variance Optimized Portfolio

Mean-variance optimization takes into account correlations between single factors in order to minimize risk while reaching a certain target return. Hence the following optimization problem is solved: 

![F4](https://i.imgur.com/DzcgjQA.png)

where $\mu$ = E[$\mathbf{R}_{t+1}$], is expected value of the $N$ x 1 vector of risky portfolio returns and  $\mathbf{\Sigma_t}$ the estimated covariance matrix conditional on the information available at time $t$. $R_f$ denotes the risk free return, $\mathbf{1}$ is a vector of ones, $\mathbf{w_t}$ a vector of weights, and  $\mu_p$ captures the target return. 

One of the varying factors in this calculation is the number of observations known at time $t$ to include to get an accurate estimate of next months covariance matrix. 

In [ ]:
# Window Size
bigWindow <- 252
mediumWindow <- 66 
smallWindow <-22

# Target returns
highTarget <- mean(data$Mkt)
midTarget <-  mean(data$`Mkt-RF`)
lowTarget <-  mean(data$RMW)

MVO_Data <- data[,c("Mkt-RF","SMB","HML","RMW","CMA","RF")]

In [ ]:
# Rebalance weights based on underlying portfolio inputs and strategy
getweights <- function(strategy, returns, last_period_n, next_period_n, window=NULL, target=NULL){

    # Get number of assets in portfolio
    m <- ncol(returns)
    
    # Minimize variance
    if(strategy == "MVarOpt"){
        #Initialize weights and leave room for risk free
        w <- rep(0,m)
        #Get last t
        n <-nrow(returns)
        # Get returns in specified window
        riskyAssets <- returns[max(n+1-window,1):n,1:(m-1)]
        # Vector of riskfrees
        riskFree <- as.numeric(returns[n,m])
        # Add riskfree to riskyassets
        riskyAssets <- riskyAssets 
        
        # Get expected. Maybe use longer window for this??
        mu <- colMeans(riskyAssets) 
        # Get covariance matrix 
        Dmat <- cov(riskyAssets)
        # Constant in objective function is 0
        dvec <- rep(0,m-1)
        # Constraints:   #1 target rate
        Amat <- matrix(c(mu,
                         #2 sum smaller than one
                         rep(-1,m-1),
                         # 3 No weight larger than 1
                         -as.vector(diag(m-1)),
                         # 4 No weight smaller than 0
                         as.vector(diag(m-1))
              ),ncol=2*m)


        # Right hand of constraint
        bvec <- as.matrix(c(target , -1, rep(-1,m-1), rep(0,m-1)))

        # If no solution to optimization program, lower target. If that fails, only invest in risk free
        solution <- tryCatch(solve.QP(Dmat, t(dvec), Amat, bvec, meq=1, factorized=FALSE)$solution,
                             error=function(e){
                               # If no solution available; must lower target! 
                               bvec <- as.matrix(c(mean(mu), 1, rep(-1,n-1), rep(0,n-1)))
                               # If that also does not work; naive strategy
                               return(tryCatch(solve.QP(Dmat, t(dvec), Amat, bvec, meq=1, factorized=FALSE)$solution,
                                               error=function(e) return(rep(0, m-1))))
                             })
        # Save solution
        w[1:(m-1)] <- solution
        # Weight for risk free asset
        w[m] <- 1-sum(w[1:(m-1)])

        return(w)
    }
}

As an example strategy, let us calculate the returns using covariance matrices constructed from the last month only and a target return equal to the mean return of the Robust minus Weak portfolio.

In [ ]:
data$MeanVarOpt_SW_LT <-portfolio_returns(MVO_Data,data$month,"MVarOpt",window=smallWindow,target=lowTarget) 
summary(data$MeanVarOpt_SW_LT)

## Interlude Performances

Let's see how the simple models we have seen so far perform.

In [ ]:
models <- c("Mkt-RF","SMB","HML","RMW","CMA","Naive","VolMan","MeanVarOpt_SW_LT")
getPerfomances(models)

# 5. GARCH Theory: <a class="anchor" id="5-bullet"></a>

## Three Specifications

### GARCH(1 , 1)

Generalized Autoregressive Conditional Heteroskedasticity modelling in its simpelest form fits parameters to a formula that predicts the one-day-ahead variance based on a constant, a mean-reverting factor and an error term. The most common specification, the GARCH(1,1) model, assumes the market shock $\varepsilon_t = (r_t - \mu)/\sigma_t$ to be normally distributed. The modeled variance of each factor at time $t+1$ is calculated using various parameters via the GARCH-filter:

![F5](https://i.imgur.com/P6Ritpk.png)

### GAS Garch

One assumption made by **(5)** that is very likely to be violated is the normal distribution of its error terms. Taking the square of the error term in its model makes it very susceptible to outlier bias. One proposed solution is the [Generalized Autoregressive Score](http://www.gasmodel.com/background.htm), which replaces our specification as follows:

![F6](https://i.imgur.com/tf0lky4.png)

### Generalized T Distribution

Andrew Harvey and Rutger-Jan Lange proposed another solution to the above stated problem. In their [paper](http://www.econ.cam.ac.uk/research-files/repec/cam/pdf/cwpe1517.pdf), the derive the following GARCH filter:

![F7](https://i.imgur.com/WORDSnw.png)

where $K(\nu,\gamma) = \nu^{\frac{1}{\gamma}} \sqrt{\frac{\Gamma(\frac{3}{\gamma}) \Gamma(\frac{\nu-2}{\gamma})}{\Gamma(\frac{1}{\gamma})\Gamma(\frac{\nu}{\gamma})}}, \quad \nu>2$.

It has been derived from the generalized T distribution:

![F8](https://i.imgur.com/Mfkc9do.png)

In [ ]:
# Garch Filter calculates all sigmas in the previous period given some parameter inputs
GarchFilter <- function(par,returns, strategy = NULL){
    # Extract the sample size 
    n <- NROW(returns)

    # Initialize sigmas
    sig2 <- rep(0,n)

    # Name parameters
    mu    <- par[1]
    omega <- par[2]
    alpha <- par[3]
    beta  <- par[4]
    
    # Unconditional Variance is omega/(1-alpha-beta)
    uncond2 <- omega/(1-alpha-beta)
    
    # First sigma is unconditional variance
    sig2[1] <- uncond2
    for(t in 2:n){
        # Get oneday aheads step by step
        sig2[t] <- onedayahead(par, lastSig2 = sig2[t-1] , lastReturn = returns[t-1], uncond2 = uncond2, strategy = strategy)
    }
    
  return(unlist(sig2))
}

# One day head forecast given the parameters, previous observations and the unconditional variance
onedayahead <- function(par, lastSig2, lastReturn, uncond2, strategy){

    # Name parameters
    mu    <- par[1]
    omega <- par[2]
    alpha <- par[3]
    beta  <- par[4]

    if(strategy == "GARCH"){
        shock2 <- (lastReturn-mu)^2
        # Get one day ahead
        onedayahead <- omega + alpha*shock2 + beta*lastSig2
    }

    if(strategy == "GARCH_Gas"){
        # Get inverse of degrees of freedom
        v_1 <- 1/par[5]
        # Squared error
        e_t2 <- ((lastReturn-mu)/lastSig2)^2
        onedayahead <- uncond2 + alpha *  #(1+3*par[5]) * 
          ( (v_1+1)/(v_1+(1-2*v_1) / e_t2 ) - 1) * lastSig2 + (alpha + beta) * (lastSig2 - uncond2)
    }

    if(strategy == "GARCH_GenT"){
        # Degrees of freedom
        v <- par[5]
        #Gamma
        y <- par[6]
        # Last days shock
        shock <- abs((lastReturn-mu) / sqrt(lastSig2))
        # Get one day ahead
        onedayahead <- uncond2 + alpha * #(2/par[5]) * (1+(par[5]+1)/par[6]) * 
          ( (1/v +1)/(1/v + (K(v,y)^(-y))/shock) -1) * lastSig2 + (alpha + beta) * (lastSig2-uncond2)
    }
    return(onedayahead)
}

where $\omega$, $\alpha$, $\beta > 0$ ensure non-negative volatility estimates. 



## Maximum Likelihood 

### GARCH Parameter Estimates

The parameters $\mathbf{\theta}$ = ($\omega, \alpha, \beta, \mu$)$'$ are re-estimated by maximizing the following log-likelihood function:

![F9](https://i.imgur.com/QZBWEt4.png)

This procedure finds the optimal values of the parameters ${\theta}$ using a rolling window of $T$ days. The length of the rolling window $T$ can be altered depending on the desired time frame considered for forecasting future volatility. 

### GAS-GARCH Parameter Estimates

Since the errors are distibuted differently, the log-likelihood function can be derived to now look as follows:

![F10](https://i.imgur.com/dJOuBwo.png)

### E-GARCH Parameter Estimates

Similarly, the Generalized T Distribution boils the formula down to another maximum likelhood specification:

![F11](https://i.imgur.com/L1tMEWF.png?1)

In [ ]:
# Function to minimize.
NegativeLogLikelihood <- function(par,returns, strategy = NULL){

    # Name parameters
    mu    <- par[1]
    omega <- par[2]
    alpha <- par[3]
    beta  <- par[4]
    
    # Calculate sigmas for given parameters
    sig2 <- GarchFilter(par,returns,strategy)
    
    if(strategy == "GARCH" | strategy == "GARCH_Asym"){
        LL <- - .5 * log(2*pi) - .5 * log(sig2) - ((returns - mu)^2) / (2*sig2)
    }
    
    if(strategy =="GARCH_Gas"){
        v <- par[5]
        LL <- - .5 * log(sig2) + gammaln((v+1)/2) -  .5 * log(v*pi) - gammaln(v/2) - ((v+1)/2)  *   log(1 + ((returns - mu)^2) / (sig2 * v))
    }

    if(strategy == "GARCH_GenT"){
        v <- par[5]
        y <- par[6]
        shock <- abs((returns - mu) / sqrt(sig2))
        LL <- - .5 * log(sig2) + log(y/(2*(v^(1/y)))) + log(K(v,y)) - (gammaln(1/y) +  gammaln(v/y) - gammaln((v+1)/y)) - 
                ((v+1)/y) * log(1 + (K(v,y)^y) * (shock^y) / v)
    }
    
    # Get negative sum
    out <- -sum(LL)
    
    # In these cases, something invalid happened (negative volatility probably). 
    # To help the optimization algorithm a bit, give it a huuuge penalty
    if(is.infinite(out) | is.nan(out) | (alpha+beta)>=.99) out <- 9999999999999999999

    return(out)
}

### Parameter significance
Finally, it is also possible to determine the standard errors of our parameter coefficient estimates. Although not further used in this research, it can be used to determine the significance of parameter estimates. The standard errors are approximated by the square root of the diagonals of the inversely approximated finite Hessian.

In [ ]:
# Finite hessian
fdhess6 <- function(par, returns, fun, strategy){
  eps <- 1e-08
  p <- length(par)
  fx <- do.call(fun,list(par=par, returns=returns,strategy=strategy))
  
  # Compute the stepsize (h)
  h <- (eps^(1/3))*pmax(abs(par),1e-3)
  xh <- x+h
  h <- xh-x
  ee <- sparseMatrix(i=1:p,j=1:p,x=h,dims=c(p,p))
 
  # Compute forward step 
  g = rep(0,p)
  foreach(i = 1:p) %dopar% {
    # Add small increments in all directions
    g[i] = do.call(fun,list(par=par+ee[,i],returns=returns,strategy=strategy))
  }
 
  ht <- matrix(h, ncol = 1)
  
  H <- ht %*% h
  
  # Compute "double" forward step 
  foreach(i = 1:p) %dopar%{
    foreach(j = i:p) %dopar%{
      H[i,j] = (do.call(fun,list(par=par+ee[,i]+ee[,j],returns=returns,strategy=strategy))-g[i]-g[j]+fx)/H[i,j]
      # Mirror
      H[j,i] = H[i,j]
    }
  }
  
  return(H)
}

K <- function(v,y){
  first <- v^(1/y)
  top <- gamma(3/y)*gamma((v-2)/y)
  bottom <- gamma(1/y)*gamma(v/y)
  return(first * sqrt(top/bottom))
}


## Demonstration 

Putting it all together in code, the process looks as follows. For given returns, optimize the maximum likelihood of possible parameter inputs to get the most likely estimates. The optimization function [optim](http://stat.ethz.ch/R-manual/R-devel/library/stats/html/optim.html) allows for lower- and upper-bound limitations on our parameters. The function takes starting values aas input. Note that the starting values given are already tuned to lead to fast convergion. Finally, there is a scaling parameter which allows to differ the step-size estimates per parameters, since some are expected to be optimized over a larger range.

In [ ]:

# Calculate garch parameters and possibly their ML approximated standard deviation
getParams <- function(returns, strategy, indices = 1:length(returns)){
    
    # Take the returns from a particular index range
    returns <- returns[indices]

    # Set solver starting values
    if(strategy == "GARCH"){
        # Parameter lower bound and upper bound
        lowerbound <- c(-99,0,0,0)
        upperbound <- c(99,5,1,1)
        # Give starting value to optimization
        startingvalues <- c(mean(returns),var(returns)/20, .1, 0.85)
        # Give scale metric for appropriate stepsizes
        parscale <- c(1,.01,.01,.01)
    }
    
    if(strategy == "GARCH_Gas"){
        # Add v
        lowerbound <- c(-99,0,0,0,2)
        upperbound <- c(99,5,1,1,999)
        startingvalues <- c(mean(returns),var(returns)/20, .1, 0.85, 5)
        parscale <- c(1,.01,.01,.01,100)
    }
    if(strategy == "GARCH_GenT"){
         # Add v
        lowerbound <- c(-99,0,0,0,2,0)
        upperbound <- c(99,5,1,1,999,5)
        startingvalues <- c(mean(returns),var(returns)/20, .1, 0.85, 5, .9)
        parscale <- c(1,.01,.01,.01,100,.5)
     }

    # Optimize using optim
    par <- suppressWarnings( 
                optim(par = startingvalues, fn = NegativeLogLikelihood, returns = returns, 
                      lower = lowerbound, upper = upperbound,strategy=strategy, method="L-BFGS-B",
                        control = list(parscale=parscale))$par
            )
    
    # Calculate sigmas for given parameters
    sig2 <- GarchFilter(par = par,returns = returns, strategy = strategy)
   
    # Calculate approximated hessian and from that the standard deviations
    ML_sd <- tryCatch(
                suppressWarnings( sqrt(abs(diag( solve( 
                        fdhess6(f = "NegativeLogLikelihood",par=par,returns = returns,strategy = strategy)))))),
                        error=function(e){  return(rep(NA, length(par))) })

    return(list(par = par, ML_sd = ML_sd, sigmas = sig2))
}

As an example, observe the parameter forecasts for the three different GARCH specifications using 300 observations.

##### GARCH Parameters

In [ ]:
params <- getParams(data$`Mkt-RF`,strategy="GARCH",indices = 100:400)
print(params$par)

##### GAS GARCH Parameters

In [ ]:
params <- getParams(data$`Mkt-RF`,strategy="GARCH_Gas",indices = 100:400)
print(params$par)

##### Generalized T - GARCH Parameters

In [ ]:
params <- getParams(data$`Mkt-RF`,strategy="GARCH_GenT",indices = 100:400)
print(params$par)

# 6. GARCH Performance: <a class="anchor" id="6-bullet"></a>

When the parameters for a GARCH specification according to the Maximum Likelihood maximalisation has been found, the strategy is to assign weights inversely proportional to expected variance. GARCH based strategies models volatility rather than using historical data. Using a GARCH specification, the forecasted volatility of each factor over the entire period is the sum of multiple one-day ahead forecasts. Since future shocks are assumed to be normally distributed, the total forecasted volatility over a m-day period would be as follows:

![F12](https://i.imgur.com/k6rBp89.png)

In [ ]:
# onedayahead is one day ahead volatility
predictVol <- function(par, sigmas, lastReturn, next_period_n, strategy = NULL){
     
    # Extract parameters
    mu    <- par[1]
    omega <- par[2]
    alpha <- par[3]
    beta  <- par[4]
 
    # Last sigma
    lastSig2 <- sigmas[length(sigmas)] 
    
    # Calculate unconditional
    uncond2 <- omega/(1-alpha-beta)
  
    # Calculate oneday ahead
    oneahead <- onedayahead(par = par, lastSig2 = lastSig2, lastReturn = lastReturn, uncond2 = uncond2, strategy = strategy)

    # Formula x
    pred <- next_period_n * uncond2 + ((1-(alpha+beta)^(next_period_n))/(1-alpha-beta)) * (oneahead-uncond2)

  return(pred)
}

This gives all the ingredients to provide a new weighting function for our portfolio creation. The weights are set as follows. First, the opimized parameters for the given GARCH specification are calculated. Then, using these parameters, the approximated sigmas, last observed return and days in period to be forecasted, the total expected volatility per asset class is determined. The weights are then set inversely to this modelled volatility forecast.

In [ ]:
# Rebalance weights based on underlying portfolio inputs and strategy
getweights <- function(strategy, returns, last_period_n, next_period_n, window=NULL, target=NULL){

    # Get number of assets in portfolio
    m <- ncol(returns)
    n <- nrow(returns)
    #Initialize weights
    w <- rep(0,n)
    
    if(substr(strategy,1,5) == "GARCH"){
        # For each column calculate parameters
        foreach(i = 1:m) %dopar%{
            # Calculate parameters
            parameters <- getParams(returns[[i]], strategy= strategy, indices = max(1,n+1-window):n)
     
            # Predict next month's volatility
            totalVol <- predictVol(par = parameters$par, sigmas = parameters$sigmas, lastReturn = returns[n,i], 
                                   next_period_n = next_period_n, strategy = strategy)
            
            #Formula
            w[i] <- 1/(totalVol)
        }
        return(w/abs(sum(w)))
  }
}

Now, let us compute the returns generated from GARCH based rebalancing strategies for our three GARCH specifications. The rolling window is set to 252, which should be about the number of working days in a year.

##### GARCH Strategy

In [ ]:
data$Garch252 <- portfolio_returns(inputData,data$month,"GARCH", data$ndays, window = 252) 
summary(data$Garch252)

##### GAS GARCH Strategy

In [ ]:
data$GasGarch252 <- portfolio_returns(inputData,data$month,"GARCH_Gas", data$ndays, window = 252) 
summary(data$GasGarch252)

##### Generalized T - GARCH Strategy

In [ ]:
data$GenTGarch252 <- portfolio_returns(inputData,data$month,"GARCH_GenT", data$ndays, window = 252) 
summary(data$GenTGarch252)

## FINAL RESULTS

In [ ]:
models <- c("Mkt-RF","SMB","HML","RMW","CMA","Naive","VolMan","MeanVarOpt_SW_LT","Garch252","GasGarch252","GenTGarch252")
getPerfomances(models)